In [1713]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

np.set_printoptions(precision=4, suppress=True)

# 1. Simple linear regression

In [1714]:
age = np.array([5, 6, 7, 8, 9])
height = np.array([100, 105, 108, 112, 115])

Calculate $\bar{x}$ (Mean Age)  
Calculate $\bar{y}$ (Mean Height)

In [1715]:
age.mean()
height.mean()

np.float64(7.0)

np.float64(108.0)

$x - \bar{x}$  
$y - \bar{y}$

In [1716]:
age - age.mean()
height - height.mean()

array([-2., -1.,  0.,  1.,  2.])

array([-8., -3.,  0.,  4.,  7.])

$(x - \bar{x})^2$

In [1717]:
(age - age.mean()) ** 2

array([4., 1., 0., 1., 4.])

$(x - \bar{x})(y - \bar{y})$

In [1718]:
(age - age.mean()) * (height - height.mean())

array([16.,  3.,  0.,  4., 14.])

Sum the $(x - \bar{x})^2$ column. This is your Denominator ($S_{xx}$).

In [1719]:
denominator = ((age - age.mean()) ** 2).sum()
denominator

np.float64(10.0)

Sum the $(x - \bar{x})(y - \bar{y})$ column. This is your Numerator ($S_{xy}$).

In [1720]:
numerator = ((age - age.mean()) * (height - height.mean())).sum()
numerator

np.float64(37.0)

Calculate Slope $m = \frac{\text{Numerator}}{\text{Denominator}}$

In [1721]:
m = numerator / denominator
m

np.float64(3.7)

Calculate Intercept $b = \bar{y} - m\bar{x}$

In [1722]:
b = height.mean() - (m * age.mean())
b

np.float64(82.1)

Final Equation:$$Height = 3.7(Age) + 82.1$$

Let's do a prediction

For Age 10: $y = 3.7(10) + 82.1 =$ $119.1 \text{ cm}$

In [1723]:
x = 10
y = m * x + b
y

np.float64(119.1)

Let's do some more predictions

In [1724]:
for x in age:
  y = m * x + b
  print(y)

100.6
104.3
108.0
111.69999999999999
115.4


How far off we are from actual heights?

In [1725]:
for x, y in zip(age, height):
  y_ = m * x + b
  print(y, y_)

100 100.6
105 104.3
108 108.0
112 111.69999999999999
115 115.4


No need to manually iterate over samples.  
Should enjoy numpy broadcasting 😏

In [1726]:
height_pred = m * age + b
height_pred

array([100.6, 104.3, 108. , 111.7, 115.4])

Calculate residuals

In [1727]:
height - height_pred

array([-0.6,  0.7,  0. ,  0.3, -0.4])

Sum Squared Residuals (SSR)

In [1728]:
np.sum((height - height_pred) ** 2)

np.float64(1.1000000000000085)

Mean Squared Error (MSE)

In [1729]:
np.mean((height - height_pred) ** 2)

np.float64(0.2200000000000017)

Root Mean Squared Error (RMSE)

In [1730]:
np.sqrt(np.mean((height - height_pred) ** 2))

np.float64(0.46904157598234475)

$$R^2 = 1 - \frac{SS_{res}}{SS_{tot}}$$

Where:  
$SS_{res}$ (Residual Sum of Squares): $\sum (y_{true} - \hat{y})^2$ — The error our model makes.  
$SS_{tot}$ (Total Sum of Squares): $\sum (y_{true} - \bar{y})^2$ — The variation in the data itself.

In [1731]:
ss_res = np.sum((height - height_pred) ** 2)
ss_tot = np.sum((height - height.mean()) ** 2)

r2 = 1 - ss_res / ss_tot
r2

np.float64(0.9920289855072463)

# 2. Multiple linear regression (with just 1 feature)

**`X` should be a matrix (two dimensions)**

In [1732]:
age
age.shape

array([5, 6, 7, 8, 9])

(5,)

1. Either expand dimension

In [1733]:
X = np.expand_dims(age, axis=1)
X
X.shape  # 5 rows, 1 col

array([[5],
       [6],
       [7],
       [8],
       [9]])

(5, 1)

1. Or reshape (preferred)

In [1734]:
X = age.reshape(-1, 1)
X
X.shape  # 5 rows, 1 col

array([[5],
       [6],
       [7],
       [8],
       [9]])

(5, 1)

In [1735]:
bias_col = np.ones(len(age))
bias_col

array([1., 1., 1., 1., 1.])

Add bias column at the beginning. To finally have `X` like:  
Shape: $(5 \times 2)$  
Column 1: The Bias (all 1s).  
Column 2: The Age values (5, 6, 7, 8, 9).  

In [1736]:
X = np.c_[bias_col, X]
X
X.shape

array([[1., 5.],
       [1., 6.],
       [1., 7.],
       [1., 8.],
       [1., 9.]])

(5, 2)

Create $y$. This is just the Height values.  
Shape: $(5 \times 1)$

In [1737]:
y = height.reshape(-1, 1)
y
y.shape

array([[100],
       [105],
       [108],
       [112],
       [115]])

(5, 1)

The Transpose ($X^T$)  

It should be shape $(2 \times 5)$.  

In [1738]:
X.T
X.T.shape

array([[1., 1., 1., 1., 1.],
       [5., 6., 7., 8., 9.]])

(2, 5)

The Gram Matrix ($X^T X$)

Captures the "spread" (variance) of your features and how much they overlap with each other (covariance).

Gram matrix will look like this:
$$\begin{bmatrix} \text{Count}(n) & \sum x \\ - & \sum x^2 \end{bmatrix}$$

In [1739]:
X.T.shape, X.shape

((2, 5), (5, 2))

In [1740]:
X.T  # Rows of this will be multiplied to -
X  # columns of this

array([[1., 1., 1., 1., 1.],
       [5., 6., 7., 8., 9.]])

array([[1., 5.],
       [1., 6.],
       [1., 7.],
       [1., 8.],
       [1., 9.]])

Matrix Multiplication

For matrices $A \in \mathbb{R}^{m \times n}$ and $B \in \mathbb{R}^{n \times p}$, the elements of the product $C = AB$ are given by:$$C_{ij} = \sum_{k=1}^{n} A_{ik} B_{kj}$$

Explanation:

$A$ is an $m \times n$ matrix (rows $\times$ columns).

$B$ is an $n \times p$ matrix.

The resulting matrix $C$ is $m \times p$.

To find the value at row $i$, column $j$ of the result, you perform a dot product of the $i$-th row of $A$ and the $j$-th column of $B$.

In [1741]:
C = np.zeros(shape=(2, 2))

for i in range(2):
  for j in range(2):
    C[i, j] = np.dot(X.T[i], X[:, j])

C

array([[  5.,  35.],
       [ 35., 255.]])

Visualizing the dot product of rows and columns

In [1742]:
C = np.zeros(shape=(2, 2), dtype=object)

for i in range(2):
  for j in range(2):
    row = X.T[i].astype(int)
    col = X[:, j].astype(int)
    dot_viz = ' + '.join([f'{a}*{b}' for a, b in zip(row, col)])
    dot_res = np.dot(row, col)
    print(f'at pos {i}{j}: {dot_viz} = {dot_res}')

at pos 00: 1*1 + 1*1 + 1*1 + 1*1 + 1*1 = 5
at pos 01: 1*5 + 1*6 + 1*7 + 1*8 + 1*9 = 35
at pos 10: 5*1 + 6*1 + 7*1 + 8*1 + 9*1 = 35
at pos 11: 5*5 + 6*6 + 7*7 + 8*8 + 9*9 = 255


In [1743]:
gram_matrix = X.T @ X
gram_matrix
gram_matrix.shape

array([[  5.,  35.],
       [ 35., 255.]])

(2, 2)

The Moment Vector ($X^T y$)

Captures the "alignment" (correlation) between your features and the target variable.

Moment vector will look like:

$$\begin{bmatrix} \sum y \\ \sum (x \cdot y) \end{bmatrix}$$

In [1744]:
X.T.shape, y.shape

((2, 5), (5, 1))

In [1745]:
moment_vector = X.T @ y
moment_vector
moment_vector.shape

array([[ 540.],
       [3817.]])

(2, 1)

The Inverse ($(X^T X)^{-1}$)

In [1746]:
gram_matrix_inv = np.linalg.inv(gram_matrix)
gram_matrix_inv
gram_matrix_inv.shape

array([[ 5.1, -0.7],
       [-0.7,  0.1]])

(2, 2)

Finally: Solve for $\beta$

Multiply `The Inverse` by `The Moment Vector`.

$$\beta = (X^T X)^{-1} X^T y$$

In [1747]:
gram_matrix_inv.shape, moment_vector.shape

((2, 2), (2, 1))

In [1748]:
beta = gram_matrix_inv @ moment_vector
beta
beta.shape

array([[82.1],
       [ 3.7]])

(2, 1)

Once you have calculated the $\beta$ vector, you have "trained" your model.

- $\beta_0$ (Intercept): $\mathbf{82.1}$

- $\beta_1$ (Slope): $\mathbf{3.7}$

Final Equation:

$$Height = 3.7(Age) + 82.1$$

Let's make the predictions

$$\hat{y} = X_{test} \cdot \beta$$

In [1749]:
X.shape, beta.shape

((5, 2), (2, 1))

In [1750]:
y_pred = X @ beta
y_pred
y_pred.shape

array([[100.6],
       [104.3],
       [108. ],
       [111.7],
       [115.4]])

(5, 1)

Evaluation metrics

In [1751]:
y
y.shape

y_pred
y_pred.shape

array([[100],
       [105],
       [108],
       [112],
       [115]])

(5, 1)

array([[100.6],
       [104.3],
       [108. ],
       [111.7],
       [115.4]])

(5, 1)

In [1752]:
y = y.flatten()
y_pred = y_pred.flatten()

y
y.shape

y_pred
y_pred.shape

array([100, 105, 108, 112, 115])

(5,)

array([100.6, 104.3, 108. , 111.7, 115.4])

(5,)

In [1753]:
ssr = np.sum((y - y_pred)**2)
ssr

np.float64(1.0999999999999972)

In [1754]:
mse = np.mean((y - y_pred)**2)
mse

np.float64(0.21999999999999945)

In [1755]:
rmse = np.sqrt(np.mean((y - y_pred)**2))
rmse

np.float64(0.46904157598234236)

In [1756]:
ss_res = np.sum((y - y_pred)**2)
ss_tot = np.sum((y - y.mean())**2)

r2 = 1 - ss_res / ss_tot
r2

np.float64(0.9920289855072464)

# 3. Multiple linear regression (with 2 features)

We are predicting Height ($y$) based on Age ($x_1$) and Weight ($x_2$).

In [1757]:
data = np.array([
    [5,  20, 100],  # age(x1), weight(x2), height(y)
    [6,  30, 110],
    [8,  25, 115],
    [7,  40, 120],
    [4,  50, 105],
    [5,  70, 140],
])

### 3D Visualization

* **Concept**: Features map to the "ground", Target maps to "elevation".
* **Axes**:
    * **$x, y$ (Age, Weight)**: The floor plane.
    * **$z$ (Height)**: The vertical axis.
* **Camera Control**:
    * **`eye`**: Sets initial camera position $(x, y, z)$.
    * **Values**: Relative to the center $(0,0,0)$.

In [1758]:
# Create DataFrame
df = pd.DataFrame(data, columns=['Age', 'Weight', 'Height'])

# 3D Plot
fig = px.scatter_3d(df, x='Age', y='Weight', z='Height')

# Update Layout: Size & Camera
fig.update_layout(
    width=800,   # Pixel width
    height=800,  # Pixel height
    scene=dict(
        camera=dict(
            # Position of the "eye" relative to center (0,0,0)
            eye=dict(x=1.5, y=1.5, z=0.5)
        )
    )
)

Step 1 ($X$): Create the Design Matrix. Remember the Bias Trick (Column of 1s first).

In [1759]:
bias_col = np.ones(len(data))
bias_col

array([1., 1., 1., 1., 1., 1.])

In [1760]:
X_ = data[:, :-1]  # data without last (target) col
X_
X_.shape

array([[ 5, 20],
       [ 6, 30],
       [ 8, 25],
       [ 7, 40],
       [ 4, 50],
       [ 5, 70]])

(6, 2)

In [1761]:
X = np.c_[bias_col, X_]
X
X.shape

array([[ 1.,  5., 20.],
       [ 1.,  6., 30.],
       [ 1.,  8., 25.],
       [ 1.,  7., 40.],
       [ 1.,  4., 50.],
       [ 1.,  5., 70.]])

(6, 3)

Step 2 ($y$): Create the Target Vector.

In [1762]:
y = data[:, [-1]]  # data with just last col
y
y.shape

array([[100],
       [110],
       [115],
       [120],
       [105],
       [140]])

(6, 1)

The Gram Matrix ($X^T X$)

Captures the "spread" (variance) of your features and how much they overlap with each other (covariance).

Gram matrix will look like this:

$$\begin{bmatrix} \text{Count}(n) & \sum x_1 & \sum x_2 \\ - & \sum x_1^2 & \sum x_1 x_2 \\ - & - & \sum x_2^2 \end{bmatrix}$$

In [1763]:
X.T.shape, X.shape

((3, 6), (6, 3))

In [1764]:
gram_matrix = X.T @ X
gram_matrix
gram_matrix.shape

array([[    6.,    35.,   235.],
       [   35.,   215.,  1310.],
       [  235.,  1310., 10925.]])

(3, 3)

The Moment Vector ($X^T y$)

Captures the "alignment" (correlation) between your features and the target variable.

Moment vector will look like:

$$\begin{bmatrix} \sum y \\ \sum (x_1 \cdot y) \\ \sum (x_2 \cdot y) \end{bmatrix}$$

In [1765]:
X.T.shape, y.shape

((3, 6), (6, 1))

In [1766]:
moment_vector = X.T @ y
moment_vector
moment_vector.shape

array([[  690.],
       [ 4040.],
       [28025.]])

(3, 1)

The Inverse ($(X^T X)^{-1}$)

In [1767]:
gram_matrix_inv = np.linalg.inv(gram_matrix)
gram_matrix_inv
gram_matrix_inv.shape

array([[ 7.0583, -0.8313, -0.0521],
       [-0.8313,  0.1152,  0.0041],
       [-0.0521,  0.0041,  0.0007]])

(3, 3)

Finally: Solve for $\beta$

Multiply `The Inverse` by `The Moment Vector`.

$$\beta = (X^T X)^{-1} X^T y$$

In [1768]:
beta = gram_matrix_inv @ moment_vector
beta
beta.shape

array([[50.3834],
       [ 5.7989],
       [ 0.7861]])

(3, 1)

Once you have calculated the $\beta$ vector, you have "trained" your model.

- Intercept ($\beta_0$): $\mathbf{50.38}$

- Age Slope ($\beta_1$): $\mathbf{5.79}$

- Weight Slope ($\beta_2$): $\mathbf{0.78}$

The Final Equation:

$$Height = 50.38 + 5.79(\text{Age}) + 0.78(\text{Weight})$$

Interpretation:

- Base Height: A child with 0 age and 0 weight would theoretically be 50.38 cm.

- Age Factor: For every year older, they grow about 5.79 cm.

- Weight Factor: For every kg heavier, they grow about 0.78 cm.

Let's make the predictions

$$\hat{y} = X_{test} \cdot \beta$$

In [1769]:
X.shape, beta.shape

((6, 3), (3, 1))

In [1770]:
y_pred = X @ beta
y_pred
y_pred.shape

array([[ 95.1004],
       [108.7605],
       [116.4278],
       [122.4205],
       [112.8848],
       [134.406 ]])

(6, 1)

In [1771]:
y
y.shape

y_pred
y_pred.shape

array([[100],
       [110],
       [115],
       [120],
       [105],
       [140]])

(6, 1)

array([[ 95.1004],
       [108.7605],
       [116.4278],
       [122.4205],
       [112.8848],
       [134.406 ]])

(6, 1)

In [1772]:
y = y.flatten()
y_pred = y_pred.flatten()

y
y.shape

y_pred
y_pred.shape

array([100, 110, 115, 120, 105, 140])

(6,)

array([ 95.1004, 108.7605, 116.4278, 122.4205, 112.8848, 134.406 ])

(6,)

### Visualizing the Regression Plane

* **Concept**:
    * **1 Feature**: We fit a **Line** ($y = mx + b$).
    * **2 Features**: We fit a **Plane** ($z = b + w_1x + w_2y$).
* **Meshgrid (`np.meshgrid`)**:
    * Creates a grid of coordinates (like floor tiles) spanning the min/max of the data.
    * Necessary to plot a continuous surface rather than just a line.
* **The Prediction ($z$)**:
    * We calculate the predicted height ($z$) for every intersection on the grid.
    * Equation: $z = \beta_{intercept} + \beta_{age} \cdot x + \beta_{weight} \cdot y$.

In [1773]:
x_grid = np.linspace(data[:, 0].min(), data[:, 0].max(), 10)  # Age
x_grid
x_grid.shape

y_grid = np.linspace(data[:, 1].min(), data[:, 1].max(), 10)  # Weight
y_grid
y_grid.shape

array([4.    , 4.4444, 4.8889, 5.3333, 5.7778, 6.2222, 6.6667, 7.1111,
       7.5556, 8.    ])

(10,)

array([20.    , 25.5556, 31.1111, 36.6667, 42.2222, 47.7778, 53.3333,
       58.8889, 64.4444, 70.    ])

(10,)

In [1774]:
xx, yy = np.meshgrid(x_grid, y_grid)
xx.shape
yy.shape

(10, 10)

(10, 10)

In [ ]:
# Create Grid (Mesh)
x_grid = np.linspace(data[:, 0].min(), data[:, 0].max(), 10)
y_grid = np.linspace(data[:, 1].min(), data[:, 1].max(), 10)
xx, yy = np.meshgrid(x_grid, y_grid)

# Calculate Z for Grid (Plane Equation)
# z = intercept + w1*x + w2*y
zz = beta[0] + beta[1]*xx + beta[2]*yy

# Plotting
fig = go.Figure()

# Scatter Points
_ = fig.add_trace(go.Scatter3d(
    x=data[:, 0], y=data[:, 1], z=y,
    mode='markers', marker=dict(size=5, color='red'),
    name='Actual Data'
))

# Regression Plane
_ = fig.add_trace(go.Surface(
    x=xx, y=yy, z=zz,
    opacity=0.5, colorscale='blues', showscale=False,
    name='Regression Plane'
))

# Layout
fig.update_layout(
    title='3D Linear Regression (Plane)',
    scene=dict(
        xaxis_title='Age',
        yaxis_title='Weight',
        zaxis_title='Height',
        camera=dict(
          eye=dict(x=1.5, y=0.5, z=1)  # Position of the "eye" relative to center (0,0,0)
        )
    ),
    width=800, height=600,
    margin=dict(l=0, r=0, b=0, t=50)
)

Let's calculate the errors

In [1776]:
ssr = np.sum((y - y_pred) ** 2)
ssr

np.float64(126.90323480200807)

In [1777]:
mse = np.mean((y - y_pred) ** 2)
mse

np.float64(21.150539133668012)

In [1778]:
rmse = np.sqrt(np.mean((y - y_pred) ** 2))
rmse

np.float64(4.598971529991028)

In [1779]:
ss_res = np.mean((y - y_pred) ** 2)
ss_tot = np.mean((y - y.mean()) ** 2)

r2 = 1 - ss_res / ss_tot
r2

np.float64(0.873096765197992)